In [84]:
import pandas as pd
import numpy as np
import json

In [85]:
data = pd.read_csv('dataset/systems_data.csv')

In [86]:
columns = list(data.columns) # Crete a list of columns in the dataset
for col in columns[:-1]: # For all columns except throughput
    data[col] = data[col].apply(str) # Change the datatype for each column to be of type string so that there are no conflicts when performing calculations on each of the columns
data_dummy = pd.get_dummies(data) #Since all the data in categorical, this creates a boolean dummy dataframe by creating columns of all the categories for each variable. This creates a column for each bar displayed
column_dummy = list(data_dummy.columns)
on_cols = list(column_dummy[1:]) # List of bars that are turned on. Initially, all the bars are on. We start from 1st index because column_dummy[0] = Throughput
off_cols = [] # No bars are turned off initially
data_tosend = {'columns': columns[:-1]}
dataframes = {} # This variable stores all variables in the dataset and the statistics of throughput for each of these variables. It is a dict of dataframes

In [94]:
filtered_data = data_dummy # Create a new dataframe to edit the values
'''
    In the next step, a new dataframe is created storing the variable categories and 
    the Max and Min Throughput value for each category. This new dataframe is then 
    converted to a JSON dictionary and sent to the client.
'''
cur_variable = on_cols[0].split('_')[0] # Used to store the variable part of the string, for ex: Workload_Data is a string them cur_variable will store "Workload".
global_temp_df = pd.DataFrame() # Dataframe used to store the categories and their Max and Min Throughput values
for col in on_cols: # For all the bars that are currently in the "on" state
    temp = filtered_data.loc[filtered_data[col] == 1] # Get the dataframe where the current category for a given variable is 1 i.e. 'on'. This is used to plot individual bars.
    temp_var = col.split('_')[0] # Get the variable part of the string
    temp_cat = col.split('_')[1] # Get the category part of the string
    if temp_var == cur_variable: # There is no change in the variable of the col
        global_temp_df = global_temp_df.append({temp_var: temp_cat, 'Max': temp.Throughput.max(), 'Min': temp.Throughput.min()}, ignore_index=True)
        # Append a row in the global_temp_df that includes the Min and Max values for a new category of current variable
    else:
        dataframes[cur_variable] = global_temp_df # Add the value of current dataframe to the dataframes dict for storage
        # Code to create a JSON Object start -------------

        chart_df = global_temp_df.to_dict(orient='records') # Convert the dataframe to a dict
        chart_df = json.dumps(chart_df) # Create a JSON object
        data_tosend[cur_variable] = chart_df # Add the JSON object to JSON array which is sent to the client

        # JSON object code end here --------------
        cur_variable = temp_var # A new variable is detected, so change the value of cur_variable to the new variable
        temp_df = pd.DataFrame(columns=[temp_var, 'Max', 'Min']) # Create a new temporary dataframe because a new variable type is detected
        temp_df = temp_df.append({temp_var: temp_cat, 'Max': temp.Throughput.max(), 'Min': temp.Throughput.min()}, ignore_index=True) # Add the Max and Min throughput values to the temp dataframe for current category
        global_temp_df = temp_df # Change global_temp_df to this new dataframe for a new variable

# At this point, global_temp_df contains the dataframe for the last variable in the dataset. We need to add this dataset to the JSON array data_tosend

dataframes[cur_variable] = global_temp_df # Add the value of current dataframe to the dataframes dict for storage
# Code to create a JSON Object start -------------

chart_df = global_temp_df.to_dict(orient='records') # Convert the dataframe to a dict
chart_df = json.dumps(chart_df) # Create a JSON object
data_tosend[cur_variable] = chart_df # Add the JSON object to JSON array which is sent to the client

# JSON object code end here --------------
data_tosend['Max Cols'] = len(column_dummy)-1 # Total number of bars to display i.e. all columns in the dummy dataframe except throughput
data_tosend['Max Thp'] = filtered_data.Throughput.max()
data_tosend['Min Thp'] = filtered_data.Throughput.min()

In [95]:
dataframes['Workload']

,Max,Min,Workload
0,41948.00,6611.50,Data
1,17080.75,239.33,File
2,18845.00,4839.00,Mail
3,16200.33,14479.33,Web


In [99]:
# for col in on_cols:
col = 'Workload_Data'
cur_var = col.split('_')[0]
cur_cat = col.split('_')[1]
temp = filtered_data.loc[filtered_data[col] == 1]
min_thp = 10
max_thp = 100
data_tochange = dataframes[cur_var]

In [109]:
data_tochange.set_index(cur_var, inplace=True)

In [111]:
data_tochange.at[cur_cat, 'Max'] = max_thp
data_tochange.at[cur_cat, 'Min'] = min_thp

In [114]:
data_tochange.reset_index(inplace=True)

In [115]:
data_tochange.to_dict(orient='records')

[{'Max': 100.0, 'Min': 10.0, 'Workload': 'Data'},
 {'Max': 17080.75, 'Min': 239.33, 'Workload': 'File'},
 {'Max': 18845.0, 'Min': 4839.0, 'Workload': 'Mail'},
 {'Max': 16200.33, 'Min': 14479.33, 'Workload': 'Web'}]